In [1]:
## Experiment Option
from easydict import EasyDict
import torch

opt = EasyDict()
opt.dataset_series = 'company' 
opt.dataset_domain = ''
opt.subtask = 'sub1' # sub1: sentence, sub2: document(full review)
opt.task = 'category' # category, term
opt.num_classes = 3 # negative, positive, neutral, (+ conflict)
opt.max_length = 200
opt.model_name = 'kobert'
opt.pos = False
opt.lastid = False
opt.top_k = 1
opt.valset_ratio = 0.2
opt.batch_size = 16
opt.num_layers = 6
opt.num_epochs = 12
opt.runs = 5
opt.seed = 42
opt.log_step = 100
opt.patience = 5
opt.device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

print(opt.device)

cuda


In [2]:
if opt.dataset_series == 'company':
    path = 'dataset/{}_train.csv'.format(opt.dataset_series)
    path_test = 'dataset/{}_test.csv'.format(opt.dataset_series)
    
import pandas as pd

df_train = pd.read_csv(path)
df_test = pd.read_csv(path_test)

print('length of train set: {:,}'.format(len(df_train)))
print('length of test set: {:,}'.format(len(df_test)))

length of train set: 2,000
length of test set: 500


In [ ]:
from data_utils import clean_sentence, preprocess
df_train = clean_sentence(df=df_train, clean_func=preprocess)
df_test = clean_sentence(df=df_test, clean_func=preprocess)

## Test analysis
1. Load model
2. Get samples from test set and tokenize
3. Predict
4. Decode high attention words
5. Result DataFrame

#### Load Model

In [4]:
opt.model_name

'kobert'

In [5]:
from models.kobert import *

if opt.model_name == 'kobert':
    model = KoBERT(opt=opt, embed_dim=768, fc_hid_dim=128, top_k=opt.top_k, att_head='all', att_pooling='mean')

In [6]:
model.load_state_dict(torch.load('state_dict/BEST_kobert_company_preprocess5_val_acc_94.0%'))

<All keys matched successfully>

#### Get samples from test set and tokenize

In [7]:
sample_idx = [7,8, 22,24] + list(range(30,38)) + list(range(110, 118))
print(sample_idx)

[7, 8, 22, 24, 30, 31, 32, 33, 34, 35, 36, 37, 110, 111, 112, 113, 114, 115, 116, 117]


In [8]:
df_samples = df_test.iloc[sample_idx].reset_index(drop=True)
df_samples

,sentence,term,category,polarity
0,팀바팀이겠지만 기본적인 업무량이 많아 야근 필수,업무량,업무,negative
1,배울만한 동료가 많이 있었으나 최근 다수 이탈하긴함 처우가 괜찮은 편이며 조직에 따...,문화,사내문화,positive
2,제조사보다 더한 탑다운 문화 엄청 쪼아대는데 모두가 따라가는 분위기 그렇지 않으면 아웃,분위기,사내문화,negative
3,초봉이 높기 때문에 주니어 연봉이 타 회사대비 높음,연봉,급여,positive
4,성과 중심의 분위기로 인한 개인주의가 만연하여 일하는 것이 즐겁지 않습니다,분위기,사내문화,negative
5,워라밸 기대하기 힘듬,워라밸,업무,negative
6,일 많고 새로운 도전도 많아서 이 회사 다니면서 편할 생각은 하지 않는게 좋음,일,업무,negative
7,자유로운 회사 분위기 사내식당 똑똑한 동료들과의 협업,분위기,사내문화,positive
8,업무를 잘하기위해서 자기개발이 많이 요구되어 워라밸 보장이 어렵다,워라밸,업무,negative
9,개발자 대우가 좋다 쓸데 없는 프로세스에 시간을 쏟지 않게함,프로세스,업무,positive


In [9]:
from data_utils import Category_Classification_Dataset as Dataset
from kobert_tokenizer import KoBertTokenizer

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

samples = Dataset(df=df_samples, tokenizer=tokenizer, opt=opt, pos_encoding=False)

from torch.utils.data import DataLoader
sample_loader = DataLoader(dataset=samples, batch_size=30, shuffle=False)
iter_sample = iter(sample_loader).next()
print(len(iter_sample['input_ids']))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


category: True
20 samples in this dataset
20


#### Predict

In [10]:
output, top_k_idx = model(iter_sample['input_ids'].squeeze(1), iter_sample['attention_masks'].squeeze(1),
                         iter_sample['token_type_ids'].squeeze(1))
predicted = torch.argmax(output, axis=1)
labels = iter_sample['labels']

correct = (predicted == labels).tolist()
labels = labels.tolist()
predicted = predicted.tolist()

#### Decode high attention words

In [11]:
high_tokens = list()
for tokens, idx in zip(iter_sample['input_ids'].squeeze(1), top_k_idx):
    high_tokens.append(tokens.squeeze(0)[idx])

words = [tokenizer.decode(tokens) for tokens in high_tokens]

#### Result DataFrame

In [12]:
map_result = {0: 'negative', 1: 'positive', 2: 'neutral'}
predict = [map_result[i] for i in predicted]
truth = [map_result[i] for i in labels]

result_dict = {'sentence': df_samples['sentence'], 'aspect': df_samples['category'],
              'high_atts': words, 'truth': truth, 'predict': predict, 'correct': correct}

df = pd.DataFrame(result_dict)
df

,sentence,aspect,high_atts,truth,predict,correct
0,팀바팀이겠지만 기본적인 업무량이 많아 야근 필수,업무,많아,negative,negative,True
1,배울만한 동료가 많이 있었으나 최근 다수 이탈하긴함 처우가 괜찮은 편이며 조직에 따...,사내문화,문화,positive,positive,True
2,제조사보다 더한 탑다운 문화 엄청 쪼아대는데 모두가 따라가는 분위기 그렇지 않으면 아웃,사내문화,모두,negative,negative,True
3,초봉이 높기 때문에 주니어 연봉이 타 회사대비 높음,급여,높,positive,positive,True
4,성과 중심의 분위기로 인한 개인주의가 만연하여 일하는 것이 즐겁지 않습니다,사내문화,분위기,negative,negative,True
5,워라밸 기대하기 힘듬,업무,듬,negative,negative,True
6,일 많고 새로운 도전도 많아서 이 회사 다니면서 편할 생각은 하지 않는게 좋음,업무,서,negative,negative,True
7,자유로운 회사 분위기 사내식당 똑똑한 동료들과의 협업,사내문화,분위기,positive,positive,True
8,업무를 잘하기위해서 자기개발이 많이 요구되어 워라밸 보장이 어렵다,업무,어렵다,negative,negative,True
9,개발자 대우가 좋다 쓸데 없는 프로세스에 시간을 쏟지 않게함,업무,좋다,positive,positive,True


#### Samples

In [13]:
def check_att(result_df, idx):
    print('Sentence: ', result_df.iloc[idx].sentence)
    print('Aspect: ', result_df.iloc[idx].aspect)
    print('High_att_words: ', result_df.iloc[idx].high_atts)
    print('Predicted: {} | Truth: {}'.format(result_df.iloc[idx].predict, result_df.iloc[idx].truth))

In [14]:
check_att(df, 16)

Sentence:  지극히 개인주의 분위기
Aspect:  사내문화
High_att_words:  분위기
Predicted: negative | Truth: negative


In [15]:
check_att(df, 19)

Sentence:  경영진의 무능과 그들만의 세상
Aspect:  사내문화
High_att_words:  무
Predicted: negative | Truth: negative


In [16]:
check_att(df, 2)

Sentence:  제조사보다 더한 탑다운 문화  엄청 쪼아대는데 모두가 따라가는 분위기 그렇지 않으면 아웃
Aspect:  사내문화
High_att_words:  모두
Predicted: negative | Truth: negative


In [17]:
check_att(df, 3)

Sentence:  초봉이 높기 때문에 주니어 연봉이 타 회사대비 높음
Aspect:  급여
High_att_words:  높
Predicted: positive | Truth: positive
